In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 20  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 100000  # Number of samples to train on.
# Path to the data txt file on disk.
train_en = '/content/drive/MyDrive/Datasets from web/train.en'

train_vi = '/content/drive/MyDrive/Datasets from web/train.vi'


# Vectorize the data.
#input_texts = []
#target_texts = []
input_characters = set()
target_characters = set()

with open(train_en, 'r', encoding='utf-8') as f:
    ip_lines = f.read().split('\n')

with open(train_en, 'r', encoding='utf-8') as f:
    op_lines = f.read().split('\n')

len(ip_lines), len(op_lines)

(133318, 133318)

In [ ]:
considered_lines_index=[]
considered_ip_lines=[]
for i, line in enumerate(ip_lines[: min(num_samples, len(ip_lines) - 1)]):
    if len(line) > 70:
      pass
    else:
      considered_ip_lines.append(line)
      considered_lines_index.append(i)
    #input_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    #target_text = '\t' + target_text + '\n'
    #input_texts.append(input_text)
    #target_texts.append(target_text)
      for char in line:
          if char not in input_characters:

              input_characters.add(char)
op_lines_use = []
for j in considered_lines_index:
  op_lines_use.append(op_lines[j])

for line in op_lines_use:     
    for char in line:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in considered_ip_lines])
max_decoder_seq_length = max([len(txt) for txt in op_lines_use])

print('Number of samples:', min(num_samples, len(op_lines_use)))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)
input_characters, len(op_lines_use), len(considered_ip_lines)

    

Number of samples: 42942
Number of unique input tokens: 96
Number of unique output tokens: 96
Max sequence length for inputs: 70
Max sequence length for outputs: 70


([' ',
  '!',
  '#',
  '$',
  '&',
  '(',
  '+',
  ',',
  '-',
  '.',
  '/',
  '0',
  '1',
  '2',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9',
  ':',
  ';',
  '=',
  '?',
  'A',
  'B',
  'C',
  'D',
  'E',
  'F',
  'G',
  'H',
  'I',
  'J',
  'K',
  'L',
  'M',
  'N',
  'O',
  'P',
  'Q',
  'R',
  'S',
  'T',
  'U',
  'V',
  'W',
  'X',
  'Y',
  'Z',
  '_',
  'a',
  'b',
  'c',
  'd',
  'e',
  'f',
  'g',
  'h',
  'i',
  'j',
  'k',
  'l',
  'm',
  'n',
  'o',
  'p',
  'q',
  'r',
  's',
  't',
  'u',
  'v',
  'w',
  'x',
  'y',
  'z',
  'á',
  'ã',
  'è',
  'é',
  'í',
  'ñ',
  'ó',
  'ö',
  'ø',
  'ü',
  'ā',
  'ย',
  'ร',
  'อ',
  '่',
  '–',
  '—',
  '…'],
 42942,
 42942)

In [ ]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

input_token_index



{' ': 0,
 '!': 1,
 '#': 2,
 '$': 3,
 '&': 4,
 '(': 5,
 '+': 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '/': 10,
 '0': 11,
 '1': 12,
 '2': 13,
 '3': 14,
 '4': 15,
 '5': 16,
 '6': 17,
 '7': 18,
 '8': 19,
 '9': 20,
 ':': 21,
 ';': 22,
 '=': 23,
 '?': 24,
 'A': 25,
 'B': 26,
 'C': 27,
 'D': 28,
 'E': 29,
 'F': 30,
 'G': 31,
 'H': 32,
 'I': 33,
 'J': 34,
 'K': 35,
 'L': 36,
 'M': 37,
 'N': 38,
 'O': 39,
 'P': 40,
 'Q': 41,
 'R': 42,
 'S': 43,
 'T': 44,
 'U': 45,
 'V': 46,
 'W': 47,
 'X': 48,
 'Y': 49,
 'Z': 50,
 '_': 51,
 'a': 52,
 'b': 53,
 'c': 54,
 'd': 55,
 'e': 56,
 'f': 57,
 'g': 58,
 'h': 59,
 'i': 60,
 'j': 61,
 'k': 62,
 'l': 63,
 'm': 64,
 'n': 65,
 'o': 66,
 'p': 67,
 'q': 68,
 'r': 69,
 's': 70,
 't': 71,
 'u': 72,
 'v': 73,
 'w': 74,
 'x': 75,
 'y': 76,
 'z': 77,
 'á': 78,
 'ã': 79,
 'è': 80,
 'é': 81,
 'í': 82,
 'ñ': 83,
 'ó': 84,
 'ö': 85,
 'ø': 86,
 'ü': 87,
 'ā': 88,
 'ย': 89,
 'ร': 90,
 'อ': 91,
 '่': 92,
 '–': 93,
 '—': 94,
 '…': 95}

In [ ]:
input_texts = considered_ip_lines
target_texts = op_lines_use

In [ ]:
len(target_texts), len(input_texts)

(42942, 42942)

In [ ]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

encoder_input_data.shape


(42942, 70, 96)

In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.
    
encoder_input_data


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0.

In [ ]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')


Epoch 1/20
537/537 [==============================] - 13s 19ms/step - loss: 1.9834 - accuracy: 0.5175 - val_loss: 1.2732 - val_accuracy: 0.6405
Epoch 2/20
537/537 [==============================] - 10s 18ms/step - loss: 1.2207 - accuracy: 0.6535 - val_loss: 1.0768 - val_accuracy: 0.6917
Epoch 3/20
537/537 [==============================] - 10s 18ms/step - loss: 1.0446 - accuracy: 0.7006 - val_loss: 0.9614 - val_accuracy: 0.7212
Epoch 4/20
537/537 [==============================] - 10s 18ms/step - loss: 0.9446 - accuracy: 0.7276 - val_loss: 0.8994 - val_accuracy: 0.7398
Epoch 5/20
537/537 [==============================] - 10s 19ms/step - loss: 0.8774 - accuracy: 0.7458 - val_loss: 0.8373 - val_accuracy: 0.7568
Epoch 6/20
537/537 [==============================] - 10s 19ms/step - loss: 0.8219 - accuracy: 0.7612 - val_loss: 0.8083 - val_accuracy: 0.7647
Epoch 7/20
537/537 [==============================] - 10s 18ms/step - loss: 0.7841 - accuracy: 0.7710 - val_loss: 0.7733 - val_accuracy:

In [ ]:

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    #target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Rachel Pike : The science behind a climate headline
Decoded sentence: avin : This is a problem with an example of these                      
-
Input sentence: They are both two branches of the same field of atmospheric science .
Decoded sentence: o are yount at the problem that we can do this in the companies .      
-
Input sentence: That report was written by 620 scientists from 40 countries .
Decoded sentence: hate responsible , it comes in the world and so income to .            
-
Input sentence: They wrote almost a thousand pages on the topic .
Decoded sentence: o , they are all the way that they were there .                        
-
Input sentence: Over 15,000 scientists go to San Francisco every year for that .
Decoded sentence: ever , maybe we have the way they were the way they come back .        
-
Input sentence: We blow it up and look at the pieces .
Decoded sentence: e diff like a lot of things are there .                                
-
Input senten